Importing the useful libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Loading the data

## Data Analysis

## Word enbeddings matrix

### Generation

### Visualization

## Building the text classifier

### Training texts features

### Linear classifiers

### Predictions

## Output creation and visualization

### Creating the submission file

### Visualization of the results